In [1]:
import json
import csv
import datetime
import pandas as pd
import arrow
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import gspread

In [7]:
with open('message_1.json') as json_file:
    data = json.load(json_file)

logdata = data["messages"]
rem_list = ['type']

pics = []
for workout in logdata:
    if 'photos' in workout.keys():
        ts = workout.get('timestamp_ms')
        dt = arrow.Arrow.fromtimestamp(ts)
        # Asumi's default timezone for Denver
        if 'Angela Suguro' in workout.get('sender_name'):
            workout['datetime'] = dt.to('US/Central').format('MM-DD-YYYY HH:mm:ss')
            workout['w_date'] = dt.to('US/Central').format('MM-DD-YYYY')
            workout['timestamp'] = dt.to('US/Central').format('HH:mm:ss')
        # Tina's default timezone for NY
        elif 'Tina Peng' in workout.get('sender_name'):
            workout['datetime'] = dt.to('US/Eastern').format('MM-DD-YYYY HH:mm:ss')
            workout['w_date'] = dt.to('US/Eastern').format('MM-DD-YYYY')
            workout['timestamp'] = dt.to('US/Eastern').format('HH:mm:ss')
        else:
            workout['datetime'] = dt.to('US/Pacific').format('MM-DD-YYYY HH:mm:ss')
            workout['w_date'] = dt.to('US/Pacific').format('MM-DD-YYYY')
            workout['timestamp'] = dt.to('US/Pacific').format('HH:mm:ss')
        pics.append(workout)
        [workout.pop(key) for key in rem_list]

In [8]:
print(pics)

[{'sender_name': 'Alan Wong', 'timestamp_ms': 1669832010029, 'photos': [{'uri': 'messages/inbox/holiday2223fitnesschallenge_ti2qhwa1xa/photos/317056344_1522466141570180_5342699473999875700_n_1522466134903514.jpg', 'creation_timestamp': 1669832008}], 'reactions': [{'reaction': 'â\x9d¤', 'actor': 'Scott Fan'}, {'reaction': 'â\x9d¤', 'actor': 'Chris Huang'}, {'reaction': 'â\x9d¤', 'actor': 'Benson Trenh'}, {'reaction': 'â\x9d¤', 'actor': 'June Kang'}, {'reaction': 'â\x9d¤', 'actor': 'Shannon Chu'}, {'reaction': 'â\x9d¤', 'actor': 'Scott Fan'}, {'reaction': 'â\x9d¤', 'actor': 'Chris Huang'}, {'reaction': 'â\x9d¤', 'actor': 'Benson Trenh'}, {'reaction': 'â\x9d¤', 'actor': 'June Kang'}, {'reaction': 'â\x9d¤', 'actor': 'Shannon Chu'}], 'datetime': '11-30-2022 10:13:30', 'w_date': '11-30-2022', 'timestamp': '10:13:30'}, {'sender_name': 'June Kang', 'timestamp_ms': 1669813052578, 'photos': [{'uri': 'messages/inbox/holiday2223fitnesschallenge_ti2qhwa1xa/photos/317485790_524010092984329_867915198

In [9]:
# writes to csv file
data_file = open('workout_data.csv', 'w') 

csv_writer = csv.writer(data_file) 

count = 0

for workout in pics:
    if count == 0:

            header = workout.keys()
            csv_writer.writerow(header)
            count += 1
        
    csv_writer.writerow(workout.values())
data_file.close()

In [10]:
# converting csv file to panda dataframe
df = pd.read_csv('workout_data.csv', parse_dates=['datetime','w_date'])

# delete unnecessary columns
df = df.drop(['photos', 'reactions', 'timestamp_ms'], axis=1)

# starting date of workout
first_date = '11/13/2022'
df['startdate'] = pd.to_datetime(first_date, format="%m/%d/%Y")

# calculate week # based on start date
df['weeknumber'] =  (((df['w_date'] - df['startdate']).dt.days)/7)+1
df['weeknumber'] =  df['weeknumber'].astype(int)

# audit timestamp
upl_date = datetime.datetime.now()
df['upload_date'] = pd.to_datetime(upl_date)

# weekly goal
df.loc[df['weeknumber'] == 1, 'week_goal'] = 2
df.loc[df['weeknumber'] > 1, 'week_goal'] = 3


df['workout_count'] = 1

df=df.sort_values(by='datetime')


In [11]:
print(df)

  sender_name            datetime     w_date timestamp  startdate  weeknumber  \
1   June Kang 2022-11-30 04:57:32 2022-11-30  04:57:32 2022-11-13           3   
0   Alan Wong 2022-11-30 10:13:30 2022-11-30  10:13:30 2022-11-13           3   

                 upload_date  week_goal  workout_count  
1 2022-11-30 14:43:32.948296        3.0              1  
0 2022-11-30 14:43:32.948296        3.0              1  


In [17]:
# time-series bins
df.loc[df['timestamp'].between('00:00:00','08:00:00'), 'time_type'] = 'early morning'
df.loc[df['timestamp'].between('08:00:00','12:00:00'), 'time_type'] = 'late morning'
df.loc[df['timestamp'].between('12:00:00','17:00:00'), 'time_type'] = 'afternoon'
df.loc[df['timestamp'].between('17:00:00','11:59:59'), 'time_type'] = 'evening'

In [18]:
print(df)

  sender_name            datetime     w_date timestamp  startdate  weeknumber  \
1   June Kang 2022-11-30 04:57:32 2022-11-30  04:57:32 2022-11-13           3   
0   Alan Wong 2022-11-30 10:13:30 2022-11-30  10:13:30 2022-11-13           3   

                 upload_date  week_goal  workout_count      time_type  
1 2022-11-30 14:43:32.948296        3.0              1  early morning  
0 2022-11-30 14:43:32.948296        3.0              1   late morning  
